# Goal: 

* Running CD-HIT priming_exp OTUs & bac_genome1210 16S rRNA gene dataset
  * cutoff 97% seqID
  * ID OTUs with taxa from both datasets
    * target genomes

# User variables

In [23]:
baseDir = '/home/nick/notebook/SIPSim/dev/priming_exp/'
workDir = os.path.join(baseDir, 'CD-HIT')
rnammerDir = '/home/nick/notebook/SIPSim/dev/bac_genome1210/rnammer/'
otuRepFile = '/var/seq_data/priming_exp/otusn.pick.fasta'
otuTaxFile = '/var/seq_data/priming_exp/otusn_tax/otusn_tax_assignments.txt'
genomeDir = '/home/nick/notebook/SIPSim/dev/bac_genome1210/genomes/'
bulkOTUs = os.path.join(baseDir, 'exp_info', 'bulk_OTU_abund_stats.txt')

# Init

In [24]:
import re
import glob
from pprint import pprint

In [25]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [26]:
%%R
library(ggplot2)
library(dplyr)
library(tidyr)
library(gridExtra)

In [27]:
if not os.path.isdir(workDir):
    os.makedirs(workDir)

# Filtering OTU rep sequences to just OTUs in bulk soil samples

* Only bulk soil OTUs used because they are the only ones with relative abundance info on the 'original' community

In [40]:
# loading bulk OTUs

bulk_OTUs = []
with open(bulkOTUs, 'rb') as iFH:
    for line in iFH:
        line = line.rstrip().split('\t')
        if line[0] != 'OTUId':
            bulk_OTUs.append(line[0])

bulk_OTUs = set(bulk_OTUs)
print 'Number of OTUs: {}'.format(len(bulk_OTUs))

Number of OTUs: 5695


In [41]:
# loading OTU reps
OTU_seq = {}
with open(otuRepFile, 'rb') as iFH:
    taxon = None
    for line in iFH:
        line = line.rstrip()
        if line.startswith('>'):
            taxon = line.lstrip('>')
            if taxon in bulk_OTUs:
                OTU_seq[taxon] = ''
            else:
                taxon = None
        else:
            if taxon is not None:
                OTU_seq[taxon] += line

print 'Number of OTU rep sequences: {}'.format(len(OTU_seq.keys()))                

Number of OTU rep sequences: 5695


In [45]:
# writing out filtered OTUs
bulk_OTU_rep = os.path.join(workDir, 'otusn.pick.bulk.fasta')

with open(bulk_OTU_rep, 'wb') as oFH:
    for taxon,seq in OTU_seq.items():
        oFH.write('>{}\n{}\n'.format(taxon, seq))

#### Notes

* Just printing exp bulk soil OTUs

# Symlinking sequences

In [46]:
# priming_exp otu rep sequences
!printf "Number of sequences: "
!cd $workDir; \
    grep -c ">" $bulk_OTU_rep   
    
# bac_genome sequences    
!cd $workDir; \
    ln -s -f $rnammerDir/bac_genome1210_16S.fna .
    
!printf "Number of sequences: "
!cd $workDir; \
    grep -c ">" $rnammerDir/bac_genome1210_16S.fna    

Number of sequences: 5695
Number of sequences: 4557


In [47]:
# combining sequences
!cd $workDir; \
    cat $bulk_OTU_rep bac_genome1210_16S.fna > ssu_all.fna
!printf "Number of sequences: "
!cd $workDir; \
    grep -c ">" ssu_all.fna

Number of sequences: 10252


# CD-HIT run

In [48]:
!cd $workDir; \
    cd-hit-est -i ssu_all.fna -o ssu_all_cdhit -c 0.97 -d 0

Program: CD-HIT, V4.6, Feb 20 2014, 09:04:54
Command: cd-hit-est -i ssu_all.fna -o ssu_all_cdhit -c 0.97 -d
         0

Started: Sun Aug 23 07:18:22 2015
                            Output                              
----------------------------------------------------------------
total seq: 10252
longest and shortest : 3286 and 362
Total letters: 9099117
Sequences have been sorted

Approximated minimal memory consumption:
Sequence        : 10M
Buffer          : 1 X 12M = 12M
Table           : 1 X 16M = 16M
Miscellaneous   : 4M
Total           : 44M

Table limit with the given memory limit:
Max number of representatives: 465868
Max number of word counting entries: 94413254

comparing sequences from          0  to      10252
..........    10000  finished       6050  clusters

    10252  finished       6285  clusters

Apprixmated maximum memory consumption: 71M
writing new database
writing clustering information
program completed !

Total CPU time 8.72


## Finding clusters with sequences from both datasets

In [49]:
inFile = os.path.join(workDir, 'ssu_all_cdhit.clstr')

tbl = {}
with open(inFile, 'rb') as inFH:
    clst_id = None
    for line in inFH:
        line = line.rstrip()
        if line.startswith('>'):
            clst_id = line.lstrip('>Cluster ')
            tbl[clst_id] = []
        else:
            tbl[clst_id].append(re.split('\t|, ', line))
            
print "Number of clusters: {}".format(len(tbl.keys()))            

Number of clusters: 6285


In [51]:
# clusters that have '>OTU'  and '>rRNA' (OTU and genome)
def shared_clust(x):
    otus = any([y[2].startswith('>OTU') for y in x])
    genomes = any([y[2].startswith('>rRNA') for y in x])    
    return otus == True and genomes == True

tbl_f = {x:y for x,y in tbl.items() if shared_clust(y)}
print "Number of clusters with OTUs and genomes: {}".format(len(tbl_f.keys()))

Number of clusters with OTUs and genomes: 152


### Getting taxonomic classification of OTUs in target clusters

In [52]:
# loading tax file
tax = {}
with open(otuTaxFile, 'rb') as inFH:
    for line in inFH:
        line = line.rstrip()
        if not line.startswith('OTU'):
            continue
        otu,cls,boot,_ = line.split('\t')
        cls = [x.lstrip(' __') for x in cls.split(';')]
        for i in range(8):
            try:
                len(cls[i])
            except IndexError:
                cls.append('Unclassified')
        tax[otu] = cls

In [53]:
def printDict(d, n=10):
    cnt = 0
    for x,y in d.items():
        pprint(x)
        print(y)
        cnt += 1
        if cnt >= n:
            break

In [54]:
printDict(tax, n=3)

'OTU.8469'
['Unassigned', 'Unclassified', 'Unclassified', 'Unclassified', 'Unclassified', 'Unclassified', 'Unclassified', 'Unclassified']
'OTU.11582'
['Unassigned', 'Unclassified', 'Unclassified', 'Unclassified', 'Unclassified', 'Unclassified', 'Unclassified', 'Unclassified']
'OTU.5687'
['Bacteria', 'Firmicutes', 'Clostridia', 'Clostridiales', 'Peptococcaceae', 'Thermincola', 'Unclassified', 'Unclassified']


In [55]:
# adding taxonomic classifications to OTUs in target clusters

for clstr,x in tbl_f.items():
    for y in x:
        ID = y[2].lstrip('>')
        ID = re.sub('\.\.\..+','', ID)
        #print 'ID: "{}"'.format(ID)
        try:
            y.append(tax[ID])
        except KeyError:
            y.append(None)                  

In [61]:
# gut check: printing classifications & genomes names 

for clstr,x in tbl_f.items():
    print 'Cluster: {}'.format(clstr)
    for y in x:
        ID = y[2].lstrip('>')
        if ID.startswith('OTU'):
            # classifications
            try:
                print ':'.join(y[3])[:100]
            except IndexError:
                print ':'.join(y[3])
        elif ID.startswith('rRNA'):
            # genome names
            try:
                print ID[:100]
            except IndexError:
                print ID

Cluster: 210
Bacteria:Proteobacteria:Gammaproteobacteria:Xanthomonadales:Xanthomonadaceae:Pseudoxanthomonas:Uncla
rRNA_CP003093_Pseudoxanthomonas_spadix_BD_a59_3414517-3416053_DIR-... *
Cluster: 664
Bacteria:Actinobacteria:Micromonosporales:Micromonosporaceae:Micromonospora:Unclassified:Unclassifie
Bacteria:Actinobacteria:Micromonosporales:Micromonosporaceae:Planosporangium:uncultured_bacterium:Un
Bacteria:Actinobacteria:Micromonosporales:Micromonosporaceae:Micromonospora:Unclassified:Unclassifie
Bacteria:Actinobacteria:Micromonosporales:Micromonosporaceae:Unclassified:Unclassified:Unclassified:
Bacteria:Actinobacteria:Micromonosporales:Micromonosporaceae:Unclassified:Unclassified:Unclassified:
rRNA_CP002162_Micromonospora_aurantiaca_ATCC_27029_2546563-2548065_DIR+... at +/97.67%
rRNA_CP002162_Micromonospora_aurantiaca_ATCC_27029_5533833-5535335_DIR-... at +/97.67%
rRNA_CP002162_Micromonospora_aurantiaca_ATCC_27029_6680542-6682043_DIR-... at +/97.67%
rRNA_CP000850_Salinispora_arenicola

__Notes:__

* At least most of the taxonomic classifications make sense for the genomes in each cluster

### Writing out a list of target genomes and their corresponding OTUs

In [64]:
def write_targets(tbl, fh):
    fh.write('\t'.join(['cluster', 'ssu_ID', 'target_genome', 'OTU', 'OTU_taxonomy']) + '\n')
    for clstr,x in tbl.items():
        # parsing cluster
        targets = []
        otus = []
        for y in x:
            ID = y[2].lstrip('>')
            ID = re.sub('\.\.\..+','', ID)        
            if ID.startswith('OTU'):
                otu = [ID, ':'.join(y[3])]
                otus.append(otu)
            elif ID.startswith('rRNA'):
                targets.append(ID)
        # writing out list
        for target in targets:
            for otu in otus:
                genome = target.lstrip('rRNA_')
                genome = re.sub('_\d+-\d+_DIR.+', '', genome)
                fh.write('\t'.join([clstr, target, genome] + otu) + '\n')

In [65]:
outFile = os.path.join(workDir, 'target_taxa.txt')
with open(outFile, 'wb') as oFH:
    write_targets(tbl_f, oFH)

#### Number of target genomes

In [66]:
!printf "Number of target genomes: "
!cd $workDir; \
    tail -n +2 target_taxa.txt | cut -f 3 | sort -u | wc -l

Number of target genomes: 329


### Get genome files with associated genome sequence names

In [67]:
# making table of genome file names & genome sequence IDs
p = os.path.join(genomeDir, '*.fasta')
genomeFiles = glob.glob(p)

assert len(genomeFiles) == 1210, 'There should be 1210 genome files'

In [68]:
fileSeq = {}
cnt = 0
for f in genomeFiles:
    fileSeq[f] = []
    with open(f, 'rb') as iFH:
        for line in iFH:
            if line.startswith('>'):
                line = line.lstrip('>').rstrip()
                fileSeq[f].append(line)
    cnt += 1
    if cnt % 100 == 0:
        sys.stderr.write('Number of files processed: {}\n'.format(cnt))

Number of files processed: 100
Number of files processed: 200
Number of files processed: 300
Number of files processed: 400
Number of files processed: 500
Number of files processed: 600
Number of files processed: 700
Number of files processed: 800
Number of files processed: 900
Number of files processed: 1000
Number of files processed: 1100
Number of files processed: 1200


In [69]:
printDict(fileSeq, n=5)

'/home/nick/notebook/SIPSim/dev/bac_genome1210/genomes/Rhizobium_etli_bv_phaseoli_str_IE4803.fasta'
['CP007641_Rhizobium_etli_bv__phaseoli_str__IE4803']
'/home/nick/notebook/SIPSim/dev/bac_genome1210/genomes/Prosthecochloris_aestuarii_DSM_271.fasta'
['CP001108_Prosthecochloris_aestuarii_DSM_271']
'/home/nick/notebook/SIPSim/dev/bac_genome1210/genomes/Phaeobacter_inhibens_DSM_17395.fasta'
['CP002976_Phaeobacter_inhibens_DSM_17395']
'/home/nick/notebook/SIPSim/dev/bac_genome1210/genomes/Paludibacter_propionicigenes_WB4.fasta'
['CP002345_Paludibacter_propionicigenes_WB4']
'/home/nick/notebook/SIPSim/dev/bac_genome1210/genomes/Thermodesulfobacterium_geofontis_OPF15.fasta'
['CP002829_Thermodesulfobacterium_geofontis_OPF15']


In [70]:
outFile = os.path.join(workDir, 'genomeFile_seqID.txt')

with open(outFile, 'wb') as oFH:
    for f,seqIDs in fileSeq.items():
        for seqID in seqIDs:
            line = '\t'.join([seqID, f])
            oFH.write(line + '\n')   

### Filtering genome file list by target genomes

In [71]:
%%bash -s "$workDir"
cd $1

grep -f <(tail -n +2 target_taxa.txt| cut -f 3) genomeFile_seqID.txt > genomeFile_seqID_filt.txt

wc -l genomeFile_seqID_filt.txt

329 genomeFile_seqID_filt.txt


### symlinking genome files (& associated files)

In [89]:
newGenomeDir = os.path.join(workDir, '../genomes')

if not os.path.isdir(newGenomeDir):
    os.makedirs(newGenomeDir)

In [90]:
inFile = os.path.join(workDir, 'genomeFile_seqID_filt.txt')

exts = ['.fasta','.fasta.2bit','.fasta.flat','.fasta.gdx','.fasta.sqlite3.db','.fasta.uni']

with open(inFile, 'rb') as iFH:
    for line in iFH:        
        seqID,fastaFile = line.rstrip().split('\t')
        assert os.path.isfile(fastaFile), '"{}" not found'.format(fastaFile)
        
        base,ext = os.path.splitext(fastaFile)
        path,fileBase = os.path.split(base)
        newBase = os.path.join(newGenomeDir, fileBase)

        for x in exts:
            if not os.path.islink(newBase + x):
                os.symlink(base + '.fasta', newBase + x)

In [91]:
!printf "Number of fasta files in genomes dir: "
!cd $newGenomeDir; \
    ls *fasta | wc -l

Number of fasta files in genomes dir: 306


#### Notes:

* Number of genome files is < number of taxa in genomeFile_seqID_filt.txt because:
  * each taxon name can correspond to 1 of multiple chromosomes in the taxon file
  
* All genomes files set
* TODO:
  * Create simulation of community

## Making a table of genome (taxon) relative abundances

* Will be used for community simulation

In [93]:
taxonMapFile = os.path.join(workDir, 'target_taxa.txt')
genomeFilterFile = os.path.join(workDir, 'genomeFile_seqID_filt.txt')
abundFile = os.path.join(workDir, '../exp_info', 'bulk_OTU_abund_stats.txt')

In [96]:
%%R -i taxonMapFile -i abundFile -i genomeFilterFile

taxonMap = read.delim(taxonMapFile, sep='\t') %>%
    select(target_genome, OTU)
taxonMap %>% nrow %>% print
taxonMap %>% head(n=3) %>% print

message('----------------')

genomeFilter = read.delim(genomeFilterFile, sep='\t', header=F) 
genomeFilter %>% nrow %>% print
genomeFilter %>% head(n=3) %>% print

message('----------------')

abund = read.delim(abundFile, sep='\t') 
abund %>% nrow %>% print
abund %>% head(n=3) %>% print

[1] 1817
                                  target_genome      OTU
1      CP003093_Pseudoxanthomonas_spadix_BD_a59   OTU.77
2 CP002162_Micromonospora_aurantiaca_ATCC_27029 OTU.2347
3 CP002162_Micromonospora_aurantiaca_ATCC_27029 OTU.1364
----------------
[1] 329
                                                       V1
1        CP007641_Rhizobium_etli_bv__phaseoli_str__IE4803
2                CP003093_Pseudoxanthomonas_spadix_BD_a59
3 CP001918_Enterobacter_cloacae_subsp__cloacae_ATCC_13047
                                                                                                         V2
1         /home/nick/notebook/SIPSim/dev/bac_genome1210/genomes/Rhizobium_etli_bv_phaseoli_str_IE4803.fasta
2               /home/nick/notebook/SIPSim/dev/bac_genome1210/genomes/Pseudoxanthomonas_spadix_BD-a59.fasta
3 /home/nick/notebook/SIPSim/dev/bac_genome1210/genomes/Enterobacter_cloacae_subsp_cloacae_ATCC_13047.fasta
----------------
[1] 5695
    OTUId total_count  mean_count median_count  

In [97]:
%%R

tbl.j = inner_join(taxonMap, genomeFilter, c('target_genome' = 'V1'))
tbl.j %>% nrow %>% print
tbl.j %>% head

[1] 1638
                                  target_genome      OTU
1      CP003093_Pseudoxanthomonas_spadix_BD_a59   OTU.77
2 CP002162_Micromonospora_aurantiaca_ATCC_27029 OTU.2347
3 CP002162_Micromonospora_aurantiaca_ATCC_27029 OTU.1364
4 CP002162_Micromonospora_aurantiaca_ATCC_27029 OTU.3475
5 CP002162_Micromonospora_aurantiaca_ATCC_27029  OTU.388
6 CP002162_Micromonospora_aurantiaca_ATCC_27029 OTU.1147
                                                                                                V2
1      /home/nick/notebook/SIPSim/dev/bac_genome1210/genomes/Pseudoxanthomonas_spadix_BD-a59.fasta
2 /home/nick/notebook/SIPSim/dev/bac_genome1210/genomes/Micromonospora_aurantiaca_ATCC_27029.fasta
3 /home/nick/notebook/SIPSim/dev/bac_genome1210/genomes/Micromonospora_aurantiaca_ATCC_27029.fasta
4 /home/nick/notebook/SIPSim/dev/bac_genome1210/genomes/Micromonospora_aurantiaca_ATCC_27029.fasta
5 /home/nick/notebook/SIPSim/dev/bac_genome1210/genomes/Micromonospora_aurantiaca_ATCC_27029.fast

In [98]:
%%R

tbl.j2 = inner_join(tbl.j, abund, c('OTU' = 'OTUId'))
tbl.j2 %>% nrow %>% print
tbl.j2 %>% head

[1] 1638
                                  target_genome      OTU
1      CP003093_Pseudoxanthomonas_spadix_BD_a59   OTU.77
2 CP002162_Micromonospora_aurantiaca_ATCC_27029 OTU.2347
3 CP002162_Micromonospora_aurantiaca_ATCC_27029 OTU.1364
4 CP002162_Micromonospora_aurantiaca_ATCC_27029 OTU.3475
5 CP002162_Micromonospora_aurantiaca_ATCC_27029  OTU.388
6 CP002162_Micromonospora_aurantiaca_ATCC_27029 OTU.1147
                                                                                                V2
1      /home/nick/notebook/SIPSim/dev/bac_genome1210/genomes/Pseudoxanthomonas_spadix_BD-a59.fasta
2 /home/nick/notebook/SIPSim/dev/bac_genome1210/genomes/Micromonospora_aurantiaca_ATCC_27029.fasta
3 /home/nick/notebook/SIPSim/dev/bac_genome1210/genomes/Micromonospora_aurantiaca_ATCC_27029.fasta
4 /home/nick/notebook/SIPSim/dev/bac_genome1210/genomes/Micromonospora_aurantiaca_ATCC_27029.fasta
5 /home/nick/notebook/SIPSim/dev/bac_genome1210/genomes/Micromonospora_aurantiaca_ATCC_27029.fast

In [101]:
%%R -i workDir
outFile = paste(c(workDir, 'taxa_bulk-comm_abunds.txt'), collapse='/')
write.table(tbl.j2, outFile, sep='\t', quote=F, row.names=F)

#### Notes

* __USING this file for community simulation__
  * associating OTUs with genome files & abundances

### Creating a genome-file index

In [103]:
!cd $workDir; \
    tail -n +2 taxa_bulk-comm_abunds.txt | cut -f 2,3 > ../genomes/genome_index_bulkSoil_OTU.txt